# Hardware lab 1: implementing your control law / Steer by wire

In this first hardware lab we are first going to work without the passive arms of our robot and only focus on the motors. You have 2 objectives for the lab: The first one is to tune your PD gains to implement an accurate torque control on the motors. The second one is to reproduce a [steer-by-wire](https://en.wikipedia.org/wiki/Steer-by-wire) system. This should allow you to become familiar with the API of the motors


## Preliminaries
First, listen to Garry or Katy for the first few minutes of the lab while they provide some guidance on the platform.

## Communicating with the device
The DICE machines are connected to the platform using a CAN device. This is hidden within a high-level API but you can have a look at the code if you are interested in seeing how all of this works.

The API is presented in [AROMotorControlAPI.md](https://github.com/ediaro23/hardwarelab/blob/main/AROMotorControlAPI.md). The only methods that we need are `readPosition` and `applyCurrentToMotor`.

Each method has a `motorid` parameter, taking the value 1 or 2, which points to the motor that you wish to control.

Therefore, to read the position (joint angle value) or a motor, simply call:



In [ ]:
from motor_control.AROMotorControl import AROMotorControl

mc = AROMotorControl()
mc.readPosition(motorid=1)

The returned value is an angle value in degrees, between 0 and 360.


You will have observed that the API only proposes a method to send a current command, while we expect to control or robots in torque. The torque is proportional to the current through the relationship $\tau = k I $, with $\tau$ the torque, $I$ the current (in Amperes), and k a constant given by the manufacturer. In our case, the torque constant provided by the [manufacturer](https://www.myactuator.com/product-page/rmd-l-5010) is $0.16$, which trivially gives you $I = \frac{\tau}{0.16}$. We will trust the manufacturer although this might change from a motor to the other (and is yet another source of approximation).


## Question 1
Write a function that performs one iteration of a control loop: Given a desired position and a motor, read the current state and perform one step of PD control to bring the motor to the desired state. 
<mark>Important:</mark> Please make sure that you execute this or any other function in a try-catch block (as shown in [template.py](./template.py)) so that if for any reason you have to terminate your program, the motors stop. Otherwise the motors will continue to execute the last command applied to them.


## Question 2

Test this function within a loop and plot the obtained trajectory. Tune your P and D gains accordingly until you reach a desired behaviour. We are going to control the system at a frequency of 1 khz, which means that each tick should last 1 ms. As for the rununtil function in the lab, the control loop should have a structure like this one:



In [ ]:
import time
t = time.perf_counter()
N=int(10e3) #10 seconds
dt = 1. / 1e3
wait = 1. / 1e4
for i in range(N):
    t +=dt
    #run your code
    while(time.perf_counter()-t<dt):
        pass
        time.sleep(wait)

You should of course tune your gains accordingly for the rest of the labs, but this is also the occasion to voluntarily play with the values and observe what happens when you obtain oscillatory behaviours for instance. This will become interesting in particular for the next question.

## Question 3
Now, write a 30s control loop that does the following:
+ Motor 1 is configured to track the position of motor 2
+ Motor 2 is configured to track the position of motor 1

Manually mess around with the motors while the loop is running, you should feel a haptic feedback. A similar system is implemented in some of the recent cars where the steering wheel and the wheels are no longer mechanically connected.